In [1]:
import sys
sys.path.append('../')

In [9]:
# %%capture
%run common.ipynb
# importnb allows us to load the code from other notebooks
# from importnb import Notebook
# with Notebook():
#     import OfficeAutomation.common as common
#     import OfficeAutomation.electrical as electrical

ERROR:root--File `'OfficeAutomation/common.ipynb.py'` not found.


In [3]:
import crestdsl.model as crest
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [8]:
@crest.dependency(source="req_electricity_out", target="setting_in")
class ACUnit(crest.Entity, ElectricalDevice):
    """
    Setting influences how many watts are needed for operation.
    In turn, need_watt influences req_electricity_out (i.e. how much electricty should be drawn)
    
    The state is determined by the setting and whether the AC has enough electricity.
    The unit is off, if there isn't enough electricity.
    
    The output is determined by the state the AC is in 
    -> it's the need_watt (i.e. how much we request) * efficiency factor.
    
    Except when the AC is off. Then the output is always zero. 
    -> This is because AC could be e.g. in state two and set need_watt to draw 700 watt but there is not enough going in.
    -> then the output should still be 0.
    """
    setting_in = crest.Input(res_four_switch, value="off")
    output = crest.Output(res_heat_energy, value=0)
    need_watt = crest.Local(res_electricity, value=100)
    
    # fan settings
    off = current = crest.State()
    one = crest.State()
    two = crest.State()
    three = crest.State()

    trans_to_off = crest.Transition(source=[one,two,three], target=off, guard=(lambda self: self.setting_in.value == "off" or self.electricity_in.value < self.req_electricity_out.value))
    trans_to_one = crest.Transition(source=[off,two,three], target=one, guard=(lambda self: self.setting_in.value == "one" and self.electricity_in.value >= self.req_electricity_out.value))
    trans_to_two = crest.Transition(source=[off,one,three], target=two, guard=(lambda self: self.setting_in.value == "two" and self.electricity_in.value >= self.req_electricity_out.value))
    trans_to_three = crest.Transition(source=[off,one,two], target=three, guard=(lambda self: self.setting_in.value == "three" and self.electricity_in.value >= self.req_electricity_out.value))
    
    update_off = crest.Update(state=off, target=output, function=(lambda self, dt: 0))
    update_one = crest.Update(state=one, target=output, function=(lambda self, dt: self.need_watt.value*0.35))
    update_two = crest.Update(state=two, target=output, function=(lambda self, dt: self.need_watt.value*0.35))
    update_three = crest.Update(state=three, target=output, function=(lambda self, dt: self.need_watt.value*0.35))


    influence_need_to_req = crest.Influence(source=need_watt, target="req_electricity_out")
    
    @crest.influence(source=setting_in, target=need_watt)
    def influence_need_watt(value):
        if value == "off":
            return 0
        elif value == "one":
            return 500
        elif value == "two":
            return 700
        elif value == "three":
            return 900
        else:
            return 0
